##  Phase 1 -- Weed Detection

In [1]:
import os
import torch

## GPU Check

In [2]:
!nvidia-smi
#import tensorflow as tf

Wed Sep 27 17:11:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN Xp     On   | 00000000:04:00.0 Off |                  N/A |
| 23%   30C    P8     8W / 250W |   1615MiB / 12196MiB |      0%   E. Process |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN Xp     On   | 00000000:06:00.0 Off |                  N/A |
| 23%   

In [3]:
torch.cuda.empty_cache()

In [4]:
%cd /home/yolov5c/
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 064365d8683fd002e9ad789c1e91fa3d021b44f0

/home/hasan/yolov5c
Cloning into 'yolov5'...
remote: Enumerating objects: 15937, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 15937 (delta 27), reused 39 (delta 22), pack-reused 15880
Receiving objects: 100% (15937/15937), 14.66 MiB | 20.97 MiB/s, done.
Resolving deltas: 100% (10926/10926), done.
/home/hasan/yolov5c/yolov5
HEAD is now at 064365d Update parse_opt() in export.py to work as in train.py (#10789)


In [ ]:
# install dependencies as necessary
%cd  D:/yolov5c/yolov5/

!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))


## roboflow workspace

In [2]:
%cd  /home/hasan/yolov5c/yolov5/

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="hmBwY34xtrDZLCDLvFT2")
project = rf.workspace("jbnu-bf4ij").project("a1-gmrlg")
dataset = project.version(5).download("yolov5")


/home/hasan/yolov5c/yolov5
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to a1-5 in yolov5pytorch:: 100%|██████████| 4506/4506 [00:00<00:00, 4922.35it/s]


## yolov5s selection

In [3]:
#this is the model configuration we will use for our tutorial
%cd  /home/hasan/yolo5/yolov5/models/

%cat "/home/hasan/yolo5/yolov5/models/yolov5s_modified.yaml"

/home/hasan/yolo5/yolov5/models
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 1  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [28,30, 41,41, 54,54]  
  - [90,34, 44,103, 71,70]
  - [107,107, 280,84, 243,257]
  

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 3, Conv, [128, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [256, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128,3]],
   [-1, 3, GhostConv, [128, 3, 2]],  # 3-P3/8
   [-1, 6, eca_layer, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [192]],
   [-1, 1, Conv, [512, 3, 2]],  # 7-P5/32
   [-1, 3, eca_layer, [512]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 3, GhostConv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 6, eca_layer, [256]],  # 13

   [-1, 3, GhostConv, [192, 1, 1]],
   [-1, 1, nn

## train and valid zip files extract

In [68]:
import glob
import zipfile

# importing the zipfile module
from zipfile import ZipFile

# loading the temp.zip and creating a zip object
with ZipFile("/home/hasan/yolo5/yolov5/dna3-10/adata/validp1f.zip", 'r') as zObject:

    # Extracting all the members of the zip
    # into a specific location.
    zObject.extractall(
        path=("/home/hasan/yolo5/yolov5/dna3-10/adata/"))

## training

In [ ]:

# time its performance
#
%cd /home/hasan/yolo5/yolov5/
#!python -m torch.distributed.run --nproc_per_node 2 train.py --img-size 640 --batch 16 --epochs 250 --data /home/hasan/yolo5/yolov5/dna3-10/data.yaml --cfg ./models/yolov5m.yaml --weights '' --name yolov5m_results  --optimizer SGD --cache
! python  -m torch.distributed.run --nproc_per_node 2  train.py --img-size 640 --hyp "/home/hasan/yolov5c/yolov5/data/hyps/hyp.scratch-low.yaml"   --epochs 200  --batch 32  --data "/home/hasan/yolo5/yolov5/dna3-10/adata/datap1t.yaml"   --cfg  "/home/hasan/yolo5/yolov5/models/yolov5s_modified.yaml" --weights ""    --name yolov5smodfin   --cache     --device 4,5

## evaluate the model performance

In [ ]:
# eval yolov5s on custom data for 100 epochs
# time its performance# 1120 best results if trained on 800
#%%time
%cd /home/hasan/yolo5/yolov5/
!python val.py --img 640  --data "/home/hasan/yolo5/yolov5/dna3-10/adata/datap1e.yaml"      --weights "/home/hasan/yolo5/yolov5/runs/train/yolov5smodfin5/weights/best.pt"   --iou-thres 0.45 --batch 16   --device 4

## load the weights of trained model

In [5]:
##  change the path
%cd /home/hasan/yolo5/yolov5/
%ls runs/train/yolov5sp1f3/weights 

/home/hasan/yolo5/yolov5
best.pt  last.pt


## extract test images

In [76]:
import glob
import zipfile

# importing the zipfile module
from zipfile import ZipFile

# loading the temp.zip and creating a zip object
with ZipFile("/home/hasan/yolo5/testp1f.zip", 'r') as zObject:

    # Extracting all the members of the zip
    # into a specific location.
    zObject.extractall(
        path=("/home/hasan/yolo5/"))

In [ ]:
import torch
print(torch.__version__)

In [ ]:
import sys
print(sys.version)

## inference mode

In [ ]:

# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
# use the best weights!
%cd /home/hasan/yolo5/yolov5/
!python detect.py --weights "/home/hasan/yolo5/yolov5/runs/train/yolov5sp1f3/weights/best.pt"   --nosave --view-img --source "/home/hasan/yolo5/yolov5/dna3-10/adata/testp1f/images/"   --conf 0.1  --name NAME --device 3
 

In [4]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


## display images

In [ ]:
#display inference on ALL test images
#this looks much better with longer training above

import glob
from IPython.display import Image, display

for imageName in glob.glob("/home/hasan/yolo5/yolov5/runs/detect/NAME44/*.jpg"): #assuming JPG
    display(Image(filename=imageName))
    print("\n")